In [237]:
### Used this code to confirm I have upload privileges to s3

def upload_test_file_to_s3(bucket_name, object_name, region='us-west-2'):
    import boto3
    from io import BytesIO

    # Initialize S3 client
    s3_client = boto3.client('s3', region_name=region)
    
    # Create a test file in memory
    test_file = BytesIO(b'This is a test file for S3 upload.')
    
    # Try to upload the test file
    try:
        response = s3_client.upload_fileobj(test_file, bucket_name, object_name)
        print("File uploaded successfully!")
        return response
    except Exception as e:
        print("Failed to upload file:", e)

# Usage
bucket_name = 'wri-users'
object_name = 'eglen/test-upload.txt'
upload_test_file_to_s3(bucket_name, object_name, region='us-east-1')


File uploaded successfully!


In [238]:
### Tested this code using dask, was able to upload to WRI users

from dask.distributed import Client

# Setup a local Dask client
client = Client()
print(client)

import boto3
from io import BytesIO

def upload_to_s3(bucket, key, content):
    s3 = boto3.client('s3')
    file = BytesIO(content.encode())
    s3.upload_fileobj(file, bucket, key)
    return f"Uploaded {key} to {bucket}"

from dask import delayed

# Define bucket and object names
bucket_name = 'wri-users'
object_names = ['eglen/test-upload-1.txt', 'eglen/test-upload-2.txt', 'eglen/test-upload-3.txt']

# Create delayed tasks for uploading files
tasks = [delayed(upload_to_s3)(bucket_name, name, f"Contents for {name}") for name in object_names]

# Compute tasks in parallel
results = client.compute(tasks, sync=True)
print(results)

KeyboardInterrupt: 

In [ ]:
### Now testing uploading to gfw2-data - confirmed this works too

from dask.distributed import Client

# Setup a local Dask client
client = Client()
print(client)

import boto3
from io import BytesIO

def upload_to_s3(bucket, key, content):
    s3 = boto3.client('s3')
    file = BytesIO(content.encode())
    s3.upload_fileobj(file, bucket, key)
    return f"Uploaded {key} to {bucket}"

from dask import delayed

# Define bucket and object names
bucket_name = 'gfw2-data'
object_names = ['climate/AFOLU_flux_model/organic_soils/inputs/processed/dadap_density/test-upload-1.txt', 'climate/AFOLU_flux_model/organic_soils/inputs/processed/dadap_density/test-upload-2.txt', 'climate/AFOLU_flux_model/organic_soils/inputs/processed/dadap_density/test-upload-3.txt']

# Create delayed tasks for uploading files
tasks = [delayed(upload_to_s3)(bucket_name, name, f"Contents for {name}") for name in object_names]

# Compute tasks in parallel
results = client.compute(tasks, sync=True)
print(results)

In [ ]:
# Shut down the client
client.close()
print("Dask client has been shut down.")

In [300]:
tile_id = "00N_110E"

download_dict = {
    # f"{land_cover}_2000": f"{LC_uri}/composite/2000/raw/{tile_id}.tif",
    # f"{land_cover}_2005": f"{LC_uri}/composite/2005/raw/{tile_id}.tif",
    # f"{land_cover}_2010": f"{LC_uri}/composite/2010/raw/{tile_id}.tif",
    # f"{land_cover}_2015": f"{LC_uri}/composite/2015/raw/{tile_id}.tif"
    f"{land_cover}_2020": f"s3://gfw2-data/climate/AFOLU_flux_model/LULUCF/outputs/IPCC_basic_classes/2020/{tile_id}.tif",
    planted_forest_type_layer: f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/plantation_type/SDPTv2/20230911/{tile_id}_plantation_type_oilpalm_woodfiber_other.tif", # Originally from gfw-data-lake, so it's in 400x400 windows
    planted_forest_tree_crop_layer: f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/plantation_simpleType__planted_forest_tree_crop/SDPTv2/20230911/{tile_id}.tif",  # Originally from gfw-data-lake, so it's in 400x400 windows
    peat: f"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/peatlands/processed/20230315/{tile_id}_peat_mask_processed.tif",
    dadap: f"s3://gfw2-data/climate/AFOLU_flux_model/organic_soils/inputs/processed/dadap_density/dadap_{tile_id}.tif"
    }

"s3://gfw2-data/climate/carbon_model/other_emissions_inputs/peatlands/processed/20230315/00N_110E_peat_mask_processed.tif"
# Checks whether tile exists at all. Doesn't try to download chunk if the tile doesn't exist.
tile_exists = check_for_tile(download_dict, is_final)

Tile id er_other exists. Proceeding.
